In [ ]:
include("backward_pass.jl")
include("utils.jl")
include("convolution.jl")
include("graph_building.jl")
include("load_data.jl")
include("scalar_operators.jl")
include("structures.jl")


In [13]:
using MLDatasets
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x4dim = reshape(data.features, 28, 28, 1, :)
    yhot  = Flux.onehotbatch(data.targets, 0:9) 
    Flux.DataLoader((x4dim, yhot); batchsize, shuffle=true)
end

loader (generic function with 1 method)

In [14]:
function onehotbatch(data::AbstractArray{<:Integer}, labels::AbstractUnitRange{<:Integer})
    lo, hi = extrema(data)
    lo < first(labels) && error("Value $lo not found in labels")
    hi > last(labels) && error("Value $hi not found in labels")
    offset = 1 - first(labels)
    indices = UInt32.(data .+ offset)
    return Matrix(indices, length(labels))
  end

# Example usage:
labels = train_data.targets
classes = 0:9
onehot_labels = my_onehotbatch(labels, classes)
println(onehot_labels)


[0 0 0 0 0 1 0 0 0 0; 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 1 0 0 0 0 0; 0 1 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 1; 0 0 1 0 0 0 0 0 0 0; 0 1 0 0 0 0 0 0 0 0; 0 0 0 1 0 0 0 0 0 0; 0 1 0 0 0 0 0 0 0 0; 0 0 0 0 1 0 0 0 0 0; 0 0 0 1 0 0 0 0 0 0; 0 0 0 0 0 1 0 0 0 0; 0 0 0 1 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0; 0 1 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 1 0 0; 0 0 1 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 1 0; 0 0 0 0 0 0 1 0 0 0; 0 0 0 0 0 0 0 0 0 1; 0 0 0 0 1 0 0 0 0 0; 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 1; 0 1 0 0 0 0 0 0 0 0; 0 1 0 0 0 0 0 0 0 0; 0 0 1 0 0 0 0 0 0 0; 0 0 0 0 1 0 0 0 0 0; 0 0 0 1 0 0 0 0 0 0; 0 0 1 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 1 0 0; 0 0 0 1 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 1 0; 0 0 0 0 0 0 1 0 0 0; 0 0 0 0 0 0 0 0 0 1; 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 1 0 0 0 0; 0 0 0 0 0 0 1 0 0 0; 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 1 0 0; 0 0 0 0 0 0 1 0 0 0; 0 1 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 1 0; 0 0 0 0 0 0 0 1 0 0; 0 0 0 0 0 0 0 0 0 1; 0 0 0 1 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 0 0 1 0; 0 0 0 0 0 1 

Excessive output truncated after 524289 bytes.

 0 0 0 1; 0 0 0 1 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0; 0 0 0 0 0 0 1 0 0 0; 0 0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 1 0 0 0; 0 0 0 0 0 0 0 